![MOSEK ApS](https://mosek.com/files/mosek_logo_color.png )

`Fusion`: Object-Oriented API for Conic Optimization
==============================

`Fusion` is an Object-Oriented API specifically designed for for Conic Optimization. Its main features are:
  
* (almost) Seamlessly multi-language (`C#, Java, Python, Matlab`)

* Minimalistic and Safe

* Extremely Disciplined Conic Optimization

* Limited overhead

* **MOSEK** gets what you write!

* `Fusion` provides only a (large) subset of functionalities of the low-level Optimizer API.

Basic Idea
-----------------

We want to introduce an *object-oriented* interface for optimization problem of the form:

\begin{equation}
  \begin{array}{llcll}
    \min & \sum_j c_j^T x_j + \sum_j \langle C_j , X_j \rangle&&\\
    s.t.&&&\\
    & \sum_j A_j^T x_j  & \in & [l_i, u_i] & i=1,\ldots,l.\\
    & \sum_j A_j^T x_j + b_i &\in & \{\mathcal{Q}^{n_i}, \mathcal{Q}_r^{n_i}, \} & i=1,\ldots,k.\\
    & \sum_j \langle A_{ij} , X_j\rangle + B_i & \in &\mathbb{S}_+^{n_i} & i=1,\ldots,t.\\
  \end{array}
\end{equation}


where we assume dimensions match accordingly to common rules. Cones $\mathcal{K}_i^{n_i}$ are  

* $\mathbb{R}_+$ - positive orthant ,

* $\mathcal{Q}$ - Lorentz cone,

* $\mathcal{Q_r}$ - Rotated quadratic cone,

* $\mathbb{S}_+$ - PSD matrices.

By a proper choice of dimensions, cones and coefficients, we obtain LPs, CQPs and SDPs.

# Example: a simple conic quadratic problem


We want to find the minimum norm solution to a system of linear equalities, that has its components in a given range. The formal definition is:

\begin{equation}
    \begin{aligned}
    & \min \|x\|_2 \\
    & s.t.\\
    & Ax = b \\
    & l_x\leq x \leq u_x.
    \end{aligned}
\end{equation}

In conic form it reads

\begin{equation}
    \begin{aligned}
    & \min t \\
    & s.t.\\
    & t\geq \|x\|_2 \Leftrightarrow  [t, x] \in \mathcal{Q}^{n+1} \\
    & Ax = b \\
    & l_x\leq x \leq u_x.
    \end{aligned}
\end{equation}


First, let us declare some input data:

In [1]:
n = 5

A = [ [12., 20.,  0., 25., 15.],\
      [ 2.,  8., 16.,  0.,  0.],\
      [20., 20., 20., 20., 20.]]
     
b = [288.,192.,384.]

lx= [0. for i in range(n)]
ux= [1000. for i in range(n)]


A `Fusion` code starts by the definition of an optimization model:

In [2]:
from mosek.fusion import *

M = Model()

`Model` is the object the user interfaces with. 

* Model building: variables, constraints and the objective funtion are reated by `Model`

* Set/get options

* Access to the solver status

* Runs the solver.

---------------------------------------------------------------------------------------
Using `M` we declare variables $x$ and $t$ and we get the corresponding handler in return:

In [3]:
x = M.variable('x', n, Domain.inRange(lx,ux))
t = M.variable('t', 1, Domain.unbounded())

The  class called `Domain` provides all the sets needed to define conic optimization problems:

* Proper cones $\mathbb{R}_+, \mathcal{Q}, \mathcal{Q_r},$ and $\mathbb{S}_+$,

* Linear equalities and inequalities,

* Unboundness.


To define the conic quadratic constraint on the norm of $x$ we need to stack $t$ and $x$ together: this operations are provided by the class `Variable`.

In [4]:
M.constraint(Variable.vstack(t, x), Domain.inQCone())

<mosek.fusion.ConicConstraint instance at 0x7fedf3e319e0>

For the linear equalities, the matrix-vector multiplication is provided, along with other linear operators, by the `Expr` class. Here is the code:

In [5]:
le = M.constraint( Expr.mul(A,x), Domain.equalsTo(b))

Note that the `Model.constraint` method returns an object we can refer thereafter.


The objective function is what we miss:

In [6]:
M.objective(ObjectiveSense.Minimize, t)

Now we only need to run the solver and print the solution querying the `x` handler:

In [7]:
M.solve()

print 'primal solution status = ',M.getPrimalSolutionStatus()
print 'primal solution        = \n', x.level()

primal solution status =  SolutionStatus.Optimal
primal solution        = 
[-1.2462112350726904e-08, 14.399999925447373, 4.800000005447712, 4.1706478408768495e-08, -1.3557713255520233e-08]


Information about the dual can be obtained directly:

In [12]:
print 'dual solution status = ', M.getDualSolutionStatus()
print 'Ax-b dual mult.      = ', le.dual()

dual solution status =  SolutionStatus.Optimal
Ax-b dual mult.      =  [0.14471806520291086, 0.2827359317801117, -0.2103780441985366]




The **MOSEK** logo and name are trademarks of Mosek ApS.

The code is provided as-is. Compatibility with future release of **MOSEK** or the `Fusion API` are not guaranteed.

For more information contact [support_at_mosek.com](mailto:support@mosek.com)

---------------------------------------------------------------------------------------------------------------


<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">The MOSEK collection of iPython Notebook</span> by <a xmlns:cc="http://creativecommons.org/ns#" href="www.mosek.com" property="cc:attributionName" rel="cc:attributionURL">MOSEK ApS</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>.<br />Based on a work at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/MOSEK" rel="dct:source">https://github.com/MOSEK</a>.<br />Permissions beyond the scope of this license may be available at <a xmlns:cc="http://creativecommons.org/ns#" href="www.mosek.com" rel="cc:morePermissions">www.mosek.com</a>.